<a href="https://colab.research.google.com/github/patygomes/experimentosIA/blob/main/Imers%C3%A3o_IA_Alura_%2B_Google_Gemini_Aula_05_Agentes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip -q install google-genai

In [4]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [5]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [8]:
# Pergunta ao Gemini uma informação mais recente que seu conhecimento

from IPython.display import HTML, Markdown

# Perguntar pro modelo quando é a próxima imersão de IA ###############################################
resposta = client.models.generate_content(
    model=MODEL_ID,
    contents='Quando é a próxima Imersão IA com Google Gemini da Alura?'
)

# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {resposta.text}"))

Resposta:
 A Alura não tem uma data fixa para a Imersão IA com Google Gemini. A melhor forma de saber quando a próxima edição acontecerá é:

*   **Acompanhar as redes sociais da Alura:** Fique de olho nos perfis da Alura no Instagram, LinkedIn, Twitter e Facebook. Eles costumam anunciar novas imersões por lá.
*   **Verificar a página de Imersões da Alura:** Acesse regularmente a página de Imersões no site da Alura ([https://www.alura.com.br/imersao](https://www.alura.com.br/imersao)) para ver se a Imersão IA com Google Gemini foi adicionada ao calendário.
*   **Assinar a newsletter da Alura:** Ao se inscrever na newsletter, você receberá informações sobre os próximos cursos, eventos e imersões.

Assim que a data da próxima Imersão IA com Google Gemini for divulgada, você saberá por um desses canais.

In [9]:
# Pergunta ao Gemini uma informação utilizando a busca do Google como contexto

response = client.models.generate_content(
    model=MODEL_ID,
    contents='Quando é a próxima Imersão IA com Google Gemini da Alura?',
      config={"tools":[{"google_search": {}}]}
    )

# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {response.text}"))

Resposta:
 A próxima Imersão IA com Google Gemini da Alura já aconteceu. A Imersão IA mais recente ocorreu em maio de 2025. As inscrições para a imersão estavam abertas até 11 de maio de 2025. Durante a imersão, os participantes tiveram a oportunidade de aprender sobre inteligência artificial com o Gemini do Google em 5 aulas online e gratuitas.

In [10]:
# Exibe a busca
print(f"Busca realizada: {response.candidates[0].grounding_metadata.web_search_queries}")
# Exibe as URLs nas quais ele se baseou
print(f"Páginas utilizadas na resposta: {', '.join([site.web.title for site in response.candidates[0].grounding_metadata.grounding_chunks])}")
print()
display(HTML(response.candidates[0].grounding_metadata.search_entry_point.rendered_content))

Busca realizada: ['próxima Imersão IA com Google Gemini Alura']
Páginas utilizadas na resposta: youtube.com, youtube.com, youtube.com



In [11]:
# Instalar Framework ADK de agentes do Google ################################################
!pip install -q google-adk


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.

In [13]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [14]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [15]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [16]:
##########################################
# --- Agente 1: Buscador de Notícias --- #
##########################################

def agente_buscador(topico, data_de_hoje):
    buscador = Agent(
        name="agente_buscador",
        model="gemini-2.0-flash",
        description="Agente que busca informações no Google",
        tools=[google_search],
        instruction=""""
        Você é um assistente de pesquisa. A sua tarefa é usar a ferramenta de busca do Google (google_search)
        para recuperar as últimas notícias de lançamentos muito relevantes sobre o tópico abaixo. Foque no
        máximo em 5 lançamentos relevantes, com base na quantidade e entusiasmo das notícias sobre ele.
        Se um tema tiver poucas notícias ou reações entusiasmadas, é possível qe ele não seja tão relevante assim
        e pode ser substituído por outro que tenha mais.
        Esses lançamentos relevantes devem ser atuaias, de no máximo um mês antes da data de hoje.
        """
    )

    entrada_do_agente_buscador = f"Tópico: {topico}\nData de hoje: {data_de_hoje}"
    # Executa o agente
    lancamentos = call_agent(buscador, entrada_do_agente_buscador)
    return lancamentos

In [29]:
################################################
# --- Agente 2: Planejador de posts --- #
################################################
def agente_planejador(topico, lancamentos_buscados):
    planejador = Agent(
        name="agente_planejador",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente Planejador #################################################
        instruction="""
        Você é um planejador de conteúdo especialista em redes sociais. Com base na lista de lançamentos
        mais recentes e relevantes buscados, você deve:
          usar a ferramenta do Google (google_search) para criar um plano sobre
          quais são os pontos mais relevantes que poderíamos abordar em um post sobre
          cada um deles. Você também pode usar o (google_search) para encontrar mais
          informações sobre os temas e aprofundar.
          Ao final, você irá escolher o tema mais relevante entre eles com base nas suas pesquisas
          a serem abordados no post que será escrito posteriormente.
        """,
        description="Agente que planeja posts",
        tools=[google_search]
    )

    entrada_do_agente_planejador = f"Tópico:{topico}\nLançamentos buscados: {lancamentos_buscados}"
    # Executa o agente
    plano_do_post = call_agent(planejador, entrada_do_agente_planejador)
    return plano_do_post

In [30]:
######################################
# --- Agente 3: Redator do Post --- #
######################################
def agente_redator(topico, plano_de_post):
    redator = Agent(
        name="agente_redator",
        model="gemini-2.0-flash",
        instruction="""
            Você é um Redator Criativo especializado em criar posts virais para redes sociais.
            Você escreve posts para a empresa Alura, a maior escola online de tecnologia do Brasil.
            Utilize o tema fornecido no plano de post e os pontos mais relevantes fornecidos e, com base nisso,
            escreva um rascunho de post para Instagram sobre o tema indicado.
            O post deve ser engajador, informativo, com linguagem simples e incluir 2 a 4 hashtags no final.
            """,
        description="Agente redator de posts engajadores para Instagram"
    )
    entrada_do_agente_redator = f"Tópico: {topico}\nPlano de post: {plano_de_post}"
    # Executa o agente
    rascunho = call_agent(redator, entrada_do_agente_redator)
    return rascunho

In [31]:
##########################################
# --- Agente 4: Revisor de Qualidade --- #
##########################################
def agente_revisor(topico, rascunho_gerado):
    revisor = Agent(
        name="agente_revisor",
        model="gemini-2.0-flash",
        instruction="""
            Você é um Editor e Revisor de Conteúdo meticuloso, especializado em posts para redes sociais, com foco no Instagram.
            Por ter um público jovem, entre 18 e 30 anos, use um tom de escrita adequado.
            Revise o rascunho de post de Instagram abaixo sobre o tópico indicado, verificando clareza, concisão, correção e tom.
            Se o rascunho estiver bom, responda apenas 'O rascunho está ótimo e pronto para publicar!'.
            Caso haja problemas, aponte-os e sugira melhorias.
            """,
        description="Agente revisor de post para redes sociais."
    )
    entrada_do_agente_revisor = f"Tópico: {topico}\nRascunho: {rascunho_gerado}"
    # Executa o agente
    texto_revisado = call_agent(revisor, entrada_do_agente_revisor)
    return texto_revisado

In [34]:
data_de_hoje = date.today().strftime("%d/%m/%Y")

print("🚀 Iniciando o Sistema de Criação de Posts para Instagram com 4 Agentes 🚀")

# --- Obter o Tópico do Usuário ---
topico = input("❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: ")

# Inserir lógica do sistema de agentes ################################################
if not topico:
  print("Você se esqueceu de digitar o tópico")
else:
  print(f"Maravilha! Então vamos criar um post sobre novidades em {topico}")

  lancamentos_buscados = agente_buscador(topico, data_de_hoje)
  print("\n--- Resultado do Agente 1 (Buscador) ---\n")
  display(to_markdown(lancamentos_buscados))
  print("----")

  plano_de_post = agente_planejador(topico, lancamentos_buscados)
  print("\n--- Resultado do Agente 2 (Planejador) ---\n")
  display(to_markdown(plano_de_post))
  print("----")

  rascunho_gerado = agente_redator(topico, plano_de_post)
  print("\n--- Resultado do Agente 3 (Redator) ---\n")
  display(to_markdown(rascunho_gerado))
  print("----")

  texto_revisado = agente_revisor(topico, rascunho_gerado)
  print("\n--- Resultado do Agente 4 (Revisor) ---\n")
  display(to_markdown(texto_revisado))
  print("----")

🚀 Iniciando o Sistema de Criação de Posts para Instagram com 4 Agentes 🚀
❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: IA no jornalismo
Maravilha! Então vamos criar um post sobre novidades em IA no jornalismo

--- Resultado do Agente 1 (Buscador) ---



> Ok, aqui estão algumas buscas que posso fazer para encontrar lançamentos recentes e relevantes de IA no jornalismo:
> 
> 
> Com base nas minhas buscas, aqui estão alguns lançamentos e tendências recentes e relevantes na área de IA no jornalismo:
> 
> 1.  **Estudo sobre a Aplicação de Inteligência Artificial no Jornalismo em Portugal:** Um estudo promovido pelo Fundo Europeu para os Media e Informação (EMIF) e coordenado pela Universidade Nova de Lisboa visa avaliar o impacto da IA nos processos jornalísticos, identificando práticas, desafios e oportunidades. Os dados coletados servirão de base para um Livro Branco sobre IA no jornalismo em Portugal.
> 2.  **Ferramentas de IA do Google (NotebookLM e Pinpoint):** O Google oferece ferramentas de IA como o NotebookLM e o Pinpoint, projetadas para aumentar a eficiência na redação. O NotebookLM, desenvolvido a partir do Gemini, é um assistente de pesquisa personalizado, enquanto o Pinpoint recebe novas funcionalidades semanalmente.
> 3.  **Story SpinnerAI:** Integrada ao ChatGPT, essa ferramenta ajuda jornalistas a garantir que suas histórias atendam às necessidades do público. Ela usa as seis "user needs" definidas pelo editor de desenvolvimento digital da BBC como base para sugestões de ângulos para as histórias.
> 4.  **IA para automatização e análise de dados:** A IA está sendo usada para acelerar processos, analisar dados e automatizar tarefas rotineiras, como monitorar redes sociais e gerar reportagens. Ferramentas como Wordsmith e Quill transformam dados em histórias, enquanto o Google Cloud Natural Language oferece insights sobre o conteúdo.
> 5.  **Desafios éticos e a necessidade de supervisão humana:** Apesar dos benefícios, a IA no jornalismo levanta questões éticas e desafios relacionados à confiabilidade das informações e à violação de direitos autorais. A supervisão humana é fundamental para manter a integridade das informações divulgadas e evitar a disseminação de desinformação.
> 
> Além disso, vale ressaltar que um levantamento recente revelou que 56% dos jornalistas utilizam IA em suas atividades profissionais, embora uma parte deles expresse discordância com o uso da tecnologia. O público também demonstra ambivalência em relação ao uso de IA na produção de notícias, especialmente em temas sensíveis.
> 


----

--- Resultado do Agente 2 (Planejador) ---



> Ok, com base nessas informações, podemos criar um plano de conteúdo abordando os seguintes pontos:
> 
> 1.  **Estudo sobre a Aplicação de Inteligência Artificial no Jornalismo em Portugal:**
>     *   **Possível Post:** Discutir o estudo da Universidade Nova de Lisboa sobre o impacto da IA no jornalismo em Portugal.
>         *   **Perguntas para pesquisa:** Quais são os principais objetivos do estudo? Quais metodologias estão sendo utilizadas? Quais os resultados preliminares (se houver)? Qual a importância de um Livro Branco sobre IA no jornalismo?
>         *   **Foco:** Oportunidades e desafios específicos do contexto português.
> 
> 
> 2.  **Ferramentas de IA do Google (NotebookLM e Pinpoint):**
>     *   **Possível Post:** Explorar as ferramentas de IA do Google (NotebookLM e Pinpoint) para jornalistas.
>         *   **Perguntas para pesquisa:** Quais as funcionalidades específicas do NotebookLM e Pinpoint? Como jornalistas podem usar essas ferramentas no dia a dia? Existem exemplos de uso prático? Quais as vantagens e desvantagens em comparação com outras ferramentas de IA?
>         *   **Foco:** Aplicações práticas e benefícios para a eficiência do trabalho jornalístico.
> 
> 
> 3.  **Story SpinnerAI:**
>     *   **Possível Post:** Analisar como o Story SpinnerAI pode ajudar jornalistas a criar histórias que atendam às necessidades do público.
>         *   **Perguntas para pesquisa:** Como o Story SpinnerAI funciona? Quais são as seis "user needs" da BBC que a ferramenta utiliza? Existem exemplos de histórias que foram aprimoradas com o uso dessa ferramenta? Quais as limitações e possíveis vieses da ferramenta?
>         *   **Foco:** A capacidade de adaptar histórias para diferentes públicos e a garantia de relevância.
> 
> 
> 4.  **IA para automatização e análise de dados:**
>     *   **Possível Post:** Detalhar como a IA está sendo usada para automatizar tarefas e analisar dados no jornalismo.
>         *   **Perguntas para pesquisa:** Quais são as ferramentas de IA mais utilizadas para automatização e análise de dados no jornalismo (além de Wordsmith e Quill)? Como essas ferramentas transformam dados em histórias? Quais os casos de sucesso e os desafios na implementação dessas tecnologias?
>         *   **Foco:** Exemplos concretos de aplicação da IA na coleta, análise e redação de notícias.
> 
> 


----

--- Resultado do Agente 3 (Redator) ---



> 🚨 ALERTA DE SPOILER DO FUTURO DO JORNALISMO! 🚨
> 
> Já imaginou uma inteligência artificial te ajudando a garimpar dados, encontrar os ângulos mais relevantes de uma história e até personalizar a notícia para cada leitor? 🤯
> 
> A IA não é só papo de filme futurista, ela já tá transformando o jornalismo! Olha só:
> 
> 🔍 **Análise turbinada:** Ferramentas como as do Google (NotebookLM e Pinpoint) ajudam a encontrar tesouros escondidos em meio a toneladas de informação.
> 
> 🤖 **Automatização a mil:** Softwares que transformam dados brutos em notícias prontas em segundos! 
> 
> 🎯 **Foco no que importa:** Story SpinnerAI te ajuda a entender o que o público quer e a entregar histórias que realmente engajem.
> 
> E aí, preparado(a) para essa revolução? Conta pra gente nos comentários: Qual a sua maior expectativa (ou medo) sobre a IA no jornalismo? 👇
> 
> #JornalismoDoFuturo #IAnoJornalismo #Alura #InteligenciaArtificial
> 


----

--- Resultado do Agente 4 (Revisor) ---



> O rascunho está ótimo e pronto para publicar!


----
